In [18]:
import pandas as pd


start_index = 0
end_index = 50
criteria = 'gangmu_desk'
model = 'gpt-4.1-2025-04-14'
template = 'gangmu_desk'
file_path = './buffer/to_do_more.xlsx'
output_path = './buffer/gangmu_more.csv'

to_analyze = True
to_translate_comment = True
to_check = True

In [19]:

# df = pd.read_csv(file_path, sep=None, engine='python')
# print(df.columns)
# print(df.shape)

df = pd.read_excel(file_path, sheet_name='钢木桌测试')

df = df[df['好差评结果'].isna()]
df = df.sample(n=50, replace=True)
print(df.columns)
print(df.shape)

if True:
    # df = df[['评论内容', '评分', '评论时间']]
    df['英文分析'] = ''
    df['好差评结果'] = ''
    df['审查结果'] = ''

Index(['评论内容', '中文评论', '好差评结果', '评分', '评论时间', '英文分析', '人审提取情况(wenyue)',
       '人审提取准确情况(wenyue)', '人审提取覆盖情况(wenyue)', '人审提取通过情况(Tom)',
       '人审提取准确情况(Tom)', '人审提取覆盖情况(Tom)', '人审提取通过情况(Olivia)',
       '人审提取准确情况(Olivia)', '人审提取覆盖情况(Olivia)', '最终审查结果'],
      dtype='object')
(50, 16)


In [20]:
import re
import yaml
from tqdm import tqdm
from modules.agent import OpenAICommentAnalysisAgent



def extract_output(text):
    pattern = r'\{.*?\}'
    matches = re.findall(pattern, text, re.DOTALL)

    matches = [match.replace('\n', '') for match in matches]
    return matches


def remove_duplicate_values(input_dict):
    seen_values = set()  # 用于存储已经出现过的值
    output_dict = {}  # 用于存储最终的结果

    for key, value in input_dict.items():
        if value not in seen_values:
            seen_values.add(value)
            output_dict[key] = value

    return output_dict


with open('./openai_keys.yaml', "r", encoding="utf-8") as file:
    data = yaml.safe_load(file)
my_key = data['openai']['key']

comment_analyzor = OpenAICommentAnalysisAgent(openai_key=my_key, criteria=criteria, model=model, template=template)


for i in tqdm(range(start_index, min(len(df), end_index))):
    comment = df.iloc[i]['评论内容']
    if not isinstance(comment, str):
        continue

    # 分析评论
    if to_analyze:
        analysis = comment_analyzor.comment_analyze(comment)
        df.loc[i, '英文分析'] = analysis
        # print(f"分析结果: {analysis}")

        # 提取结果
        extracted_output = extract_output(analysis)
        extracted_output = ''.join(extracted_output)
        extracted_output = eval(extracted_output)
        extracted_output = remove_duplicate_values(extracted_output)
        df.loc[i, '好差评结果'] = str(extracted_output)
        # print(f"提取结果: {extracted_output}")

    # 翻译
    if to_translate_comment:
        translated_comment = comment_analyzor.translate(comment)
        df.loc[i, '中文评论'] = translated_comment
        # print(f"原: {comment}\n翻译评论: {translated_comment}")

    # 反审查
    if to_check:
        extracted_output = df.loc[i, '好差评结果']
        check_result = comment_analyzor.inspect(comment, str(extracted_output))
        df.loc[i, '审查结果'] = check_result
df.to_csv(output_path, sep='\t', index=False)

100%|██████████| 50/50 [08:03<00:00,  9.68s/it]


In [ ]:
df.to_csv(output_path, sep='\t', index=False)

In [ ]:
print(df.shape)

In [ ]:
import pandas as pd


points = {'组装难度与时间': 'Assembly difficulty & Time', '说明书质量': 'Manual Quality', '组装部件数量': 'Number of Assembly Parts', '可改配置': 'Configurability', '部件易识别度': 'Part Identifiability', '照明功能': 'Lighting Feature', '集成电源和充电端口': 'Integrated Power and Charging Ports', '存储功能': 'Storage Features', '附加功能': 'Accessory Features', 'PC主机存放': 'PC Tower Space', '键盘托盘': 'Keyboard Tray', '置物架功能': 'Shelf Space', '洞洞板功能': 'Pegboard Feature', '腿部空间': 'Leg Room', '游戏场景适配': 'Suitability for Gaming', '外观与风格': 'Appearance & Style', '颜色': 'Color', '桌面多板拼接': 'Multi-Board Desktop', '稳定性和坚固性': 'Stability & Sturdiness', '产品可靠性': 'Product Durability', '防水性': 'Water Resistance', '桌面厚度与材质': 'Desktop Quality', '五金件质量': 'Hardware Quality', '抽屉质量': 'Drawers Quality', '预钻孔对齐情况': 'Pilot Holes Alignment', '气味情况': 'Odor', '性价比': 'Value for Money', '发货速度': 'Shipping Speed', '包装质量': 'Packaging Quality', '产品完整性': 'Product Integrity', '客户服务': 'Customer Service', '售后服务': 'After-sales Service', '退换货体验': 'Return Difficulty', '线缆管理': 'Cable Management', '桌面尺寸': 'Desktop Size', '桌子形状': 'Desk Shape', '承重能力': 'Load capacity', '值得推荐': 'Recommendation', '与商品图有差距': 'Product Image/Description Mismatch'}
points = {v.lower(): k for k, v in points.items()}

file_path = './buffer/pengcong_re.csv'
save_path = './buffer/pengcong_ren.csv'
df = pd.read_csv(file_path, sep=None, engine='python')
df['好评'] = ''
df['差评'] = ''
df['中评'] = ''

for i in range(len(df)):
    ext_pts = df.iloc[i]['好差评结果']
    try:
        ext_pts = eval(ext_pts)
    except:
        print(ext_pts)
        continue
    pos = []
    neg = []
    neu = []
    for k in ext_pts.keys():
        if 'Pos' in k or 'pos' in k:
            ch = ext_pts[k]
            ch = ch.strip().lower()
            ch = points[ch]
            pos.append(ch)
        elif 'Neg' in k or 'neg' in k:
            ch = ext_pts[k]
            ch = ch.strip().lower()
            ch = points[ch]
            neg.append(ch)
        elif 'Neu' in k or 'neu' in k:
            ch = ext_pts[k]
            ch = ch.strip().lower()
            ch = points[ch]
            neu.append(ch)
    df.loc[i, '好评'] = str(pos)
    df.loc[i, '差评'] = str(neg)
    df.loc[i, '中评'] = str(neu)

df.to_csv(save_path, index=False, sep='\t')

In [23]:
sheets = ['钢木桌', '板式办公桌', '升降桌']

file_path = './buffer/desk2.xlsx'
num_wrong_points = 0
total_points = 0

def count_points(x):
    try:
        x = eval(x)
    except:
        print(f"Error evaluating: {x}")
        return 0
    return len(x.keys())


for sheet in sheets:
    df = pd.read_excel(file_path, sheet_name=sheet)
    #df = df[df['最终'] != '不通过']
    df = df.dropna(subset=['好差评结果'])

    df['num_points'] = df['好差评结果'].apply(count_points)
    #num_wrong_points += df['人审提取准确情况(Tom)'].sum()
    total_points += df['num_points'].sum()

    # # 计算准确率
    # print(f"num_wrong_points: {num_wrong_points}, total_points: {total_points}")
    # accuracy = (total_points - num_wrong_points) / total_points
    # print(f"总的准确率: {accuracy:.2%}")
print(total_points)

560
